[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dionatrafk/workload_prediction/blob/master/MLP.ipynb)

**`Artificial Neural Network (MLP-Multi Layer Perceptron)`**

**Import datasets from git**

In [0]:
!git clone --recursive https://github.com/dionatrafk/workload_prediction

**Verify dataset file**

In [0]:
#imports
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import mean_squared_error
import math
import os, sys

In [0]:
def create_dataset(dataset, lookback=1):
    dataX, dataY = [], [] # create 2 empty list

    # go through the lenght of dataset, subtract the lookback and 1. 2 steps before the end of dataset, because we predict 1 step to the future
    for i in range(len(dataset)-lookback-1):
        a = dataset[i:(i+lookback),0]
        dataX.append(a)
        dataY.append(dataset[i+lookback,0]) # get the next value
    return np.array(dataX), np.array(dataY)


In [0]:
filename = 'workload_prediction/trace60.csv'

# good practice to set the seed state, starting point
np.random.seed(7)

# Data preparation
dataset = pd.read_csv(filename, usecols=[1], header=None)
dataset = dataset.values #convert to the array
dataset = dataset.astype('float32') # convert to float

# length of our data set
training_size = int(len(dataset)*0.67)
testing_size = len(dataset)-training_size

# split the data set
train, test = dataset[0:training_size,:], dataset[training_size:len(dataset),:]


# one time step to the future
lookback = 1
trainX, trainY = create_dataset(train, lookback)
testX, testY = create_dataset(test, lookback)

In [0]:
# create the model
model_name = 'requests_MLP'
model=Sequential()
model.add(Dense(32, input_dim=lookback, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))


In [0]:
#Compilation and training
start = time.time()
model.compile(loss = 'mean_squared_error', optimizer = "adam") 

print "Compilation Time : ", time.time() - start 

start = time.time()

BATCH_SIZE = 140
NB_EPOCHS = 150
model.fit(trainX,trainY,epochs=NB_EPOCHS, batch_size=BATCH_SIZE, verbose=0) 
print "Training time : ", time.time() - start


In [0]:

# Making predictions
yhat = model.predict(trainX)
y_train = trainY

print 'Train samples: %d' %(training_size)+'\n'+'Test samples: %d' %(testing_size)+'\n'

score = mean_squared_error(y_train, yhat)
print ('Trainscore: %.2f MSE (%.2f RMSE)' %(score, math.sqrt(score)))

yhat = model.predict(testX)
y_test = testY

score = mean_squared_error(y_test, yhat)
print ('Testscore: %.2f MSE (%.2f RMSE)' %(score, math.sqrt(score)))

#print samples
print'current  predicted'
for i in range(0, 10):
    print  y_test[i], yhat[i]

In [0]:
plt.plot(yhat[-100:], label='Predicted')
plt.plot(y_test[-100:], label='Current')
plt.legend()
plt.grid()

plt.ylabel('Requests')
plt.xlabel('Samples')
plt.show()
